In [21]:
import copy
import random
import itertools

import pandas as pd
  

### Find extrema(min, max) of a function:
\begin{equation*}
\frac{x - 3}{(x + 5)} - (x + 3)(x - 5), x  \in [15, 45], x\not=5
\end{equation*}

In [22]:
def fun(x):
    assert x != 5
    return ((x - 3) / (x + 5)) - ((x + 3) * (x - 5))

values = sorted([(fun(i), i) for i in range(15, 46)], 
                    key=lambda x: x[0], 
                    reverse=True)
print(f'actual max = {values[0]}')
print(f'actual min = {values[-1]}')

actual max = (-179.4, 15)
actual min = (-1919.16, 45)


### Util operations

In [23]:
def invert_bit(bits, i):
    x = list(bits)
    x[i] = int(not int(x[i]))
    return ''.join(map(str, x))

def fill_zeros(bits, length):
    b = list(bits)
    while len(b) != length:
        b.insert(0, 0)
    return ''.join(map(str, b))

def pairwise(iterable):
    for i in range(0, len(iterable), 2):
        try:
            yield iterable[i], iterable[i + 1]
        except IndexError:
            yield iterable[i], iterable[0]

def population_df(chromosomes):
        df = pd.DataFrame(data=[ch_i for ch_i in chromosomes], columns=['chromosome'])
        df['phenotype'] = df.apply(lambda x: phenotype(x['chromosome']), axis=1) 
        df['fun_value'] = df.apply(lambda x: fun(x['phenotype']), axis=1)
        df['sel_probability'] = df['fun_value'] / df['fun_value'].sum()
        assert round(df['sel_probability'].sum(), 5) == 1
        return df


def get_best_chromosome(c1, c2):
    return c1 if fun(phenotype(c1)) < fun(phenotype(c2)) else c2

### Chromosome representation

In [24]:
def chromosome_bits(n, bits_len=None):
    if bits_len:
        return fill_zeros(bin(n)[2:], bits_len)
    return bin(n)[2:]
    
def phenotype(n):
    return int(n, base=2)

assert chromosome_bits(10) == '1010'
assert chromosome_bits(10, bits_len=5) == '01010'
assert phenotype('1010') == 10
assert phenotype(chromosome_bits(1234)) == 1234
assert chromosome_bits(45) == '101101'
assert phenotype('101101') == 45
assert phenotype(chromosome_bits(45)) == 45

### Genetic operators

* Mutation(bit_position=3)
    - 000**0**00 >> 000**1**00

In [25]:
def mutation(chromosome, 
             probability=0.2, 
             bit_position=None):
    def mutate():
        if bit_position is None:
            pos = random.randint(0, len(chromosome) - 1)
        else:
            pos = bit_position
        return invert_bit(chromosome, pos)

    assert (0 <= probability <= 1)
    
    if random.random() < probability:
        return mutate()
    return chromosome

assert mutation('1111', bit_position=1, probability=0) == '1111'
assert mutation(chromosome_bits(15), bit_position=0, probability=1) == '0111'
assert mutation(chromosome_bits(15), bit_position=1, probability=1) == '1011'
assert mutation(chromosome_bits(15), bit_position=2, probability=1) == '1101'
assert mutation(chromosome_bits(15), bit_position=3, probability=1) == '1110'

def safe_mutation(chromosome, 
                  probability=0.2, 
                  bit_position=None,
                  allow_range=range(15, 46)):
    mutated = mutation(chromosome, probability, bit_position)
    # print(f'Mutated:', phenotype(chromosome), '->', phenotype(mutated))
    if phenotype(mutated) not in allow_range:
        return chromosome       
    return mutated

def mutate_population(_population,
                      probability=0.2, 
                      bit_position=None,
                      allow_range=range(15, 46)):
    return [safe_mutation(ch, probability, bit_position, allow_range) 
            for ch in _population]

* Crossing
    * rift = 3
        - **000**000 >> 000 | 111 = 000111
        - **111**111 >> 111 | 000 = 111000
        
    * rift = 1
        - **1**111 >> 1 | 000 = 1000
        - **0**000 >> 0 | 111 = 0111

In [26]:
def crossing(chromosome1, chromosome2, rift=None, probability=0.8):
    def cross():
        assert len(chromosome1) == len(chromosome2)
        if rift is None:
            pos = random.randint(0, len(chromosome1))
        else:
            pos = rift
        assert 0 <= pos <= len(chromosome1)
        return (''.join([chromosome1[:pos], chromosome2[pos:]]),
                ''.join([chromosome2[:pos], chromosome1[pos:]]))
    
    assert (0 <= probability <= 1)
    
    if random.random() < probability:
        return cross()
    return chromosome1, chromosome2


assert crossing('000000', '111111', 3, 1) == ('000111', '111000')
assert crossing('1111', '0000', 1, 1) == ('1000', '0111')

def safe_crossing(chromosome1, 
                  chromosome2, 
                  rift=None, 
                  probability=0.8,
                  allow_range=range(15, 46)):
    c1, c2 = crossing(chromosome1, chromosome2, rift, probability)
    # print(f'Crossing:', phenotype(chromosome1), phenotype(chromosome2), 
    #       '->', phenotype(c1), phenotype(c2))
    if (phenotype(c1) not in allow_range or 
        phenotype(c2) not in allow_range):
        return chromosome1, chromosome2
    return c1, c2

def make_crossing_pairs(_population):
    indexes = list(pairwise(range(len(_population))))
    random.shuffle(indexes)
    return [(_population[i1], _population[i2]) for i1, i2 in indexes]

def cross_population(_population,
                     probability=0.8,
                     rift=None,
                     allow_range=range(15, 46)):
    pairs = make_crossing_pairs(_population)
    crossed_pairs = list(itertools.chain.from_iterable([safe_crossing(ch1, ch2, rift, probability, allow_range)
                                                        for ch1, ch2 in pairs]))
    if len(crossed_pairs) > len(_population):
        c1 = crossed_pairs.pop()
        c2 = crossed_pairs.pop()
        crossed_pairs.append(get_best_chromosome(c1, c2))
    return crossed_pairs

### Genetic selection

In [27]:
def roulette(chromosomes, survive_probabilities):
    n = len(chromosomes)
    assert n == len(chromosomes) == len(survive_probabilities)
    survived = []
    s = sorted([(ch, sp * n, round(sp * n)) 
                for ch, sp in zip(chromosomes, survive_probabilities)],
               key=lambda x: x[1])
    while sum(ch_amount for *_, ch_amount in s) > n:
        s.pop(0)
    
    while len(survived) != n:
        ch, f_ch_amount, ch_amount = s.pop()
        survived.extend([ch] * ch_amount)
        if len(s) == 0 and len(survived) < n:
            survived.extend([ch] * (n - len(survived)))
            assert len(survived) == n
    return survived

def select_next_population(df, inverse=False):
    if inverse:
        df['sel_probability'] = (1 - df['sel_probability']) / df['sel_probability'].sum()
    return roulette(df['chromosome'], df['sel_probability'])

### Initial values

In [28]:
# population contains N chromosomes 
# each chromosome consist of L bits
N = 11
L = len(chromosome_bits(45))

population = [
    chromosome_bits(32, L),
    chromosome_bits(15, L),
    chromosome_bits(20, L),
    chromosome_bits(19, L),
    chromosome_bits(30, L),
    chromosome_bits(22, L),
    chromosome_bits(21, L),
    chromosome_bits(37, L),
    chromosome_bits(40, L),
    chromosome_bits(41, L),
    chromosome_bits(19, L),
]
population1 = copy.deepcopy(population)
population2 = copy.deepcopy(population)

print('Initial population:')
print(population_df(population))
assert N == len(population) == len(population1) == len(population2)
assert L == 6

Initial population:
   chromosome  phenotype    fun_value  sel_probability
0      100000         32  -944.216216         0.116844
1      001111         15  -179.400000         0.022200
2      010100         20  -344.320000         0.042609
3      010011         19  -307.333333         0.038032
4      011110         30  -824.228571         0.101996
5      010110         22  -424.296296         0.052506
6      010101         21  -383.307692         0.047433
7      100101         37 -1279.190476         0.158297
8      101000         40 -1504.177778         0.186138
9      101001         41 -1583.173913         0.195914
10     010011         19  -307.333333         0.038032


### Genetic algorithm

##### Find min

In [29]:
for i in range(100):
    population1 = select_next_population(population_df(population1))
    population1 = cross_population(population1, 
                                  probability=0.8, 
                                  allow_range=range(15, 46))
    population1 = mutate_population(population1, 
                                   probability=0.2, 
                                   allow_range=range(15, 46))

df1 = population_df(population1)
print(df1)
print(df1[df1['sel_probability'] == df1['sel_probability'].max()].head(1))

   chromosome  phenotype    fun_value  sel_probability
0      101101         45 -1919.160000         0.105635
1      101101         45 -1919.160000         0.105635
2      101001         41 -1583.173913         0.087141
3      101011         43 -1747.166667         0.096168
4      100100         36 -1208.195122         0.066502
5      100101         37 -1279.190476         0.070409
6      101101         45 -1919.160000         0.105635
7      101101         45 -1919.160000         0.105635
8      101101         45 -1919.160000         0.105635
9      100001         33 -1007.210526         0.055439
10     101011         43 -1747.166667         0.096168
  chromosome  phenotype  fun_value  sel_probability
0     101101         45   -1919.16         0.105635


##### Find max

In [30]:
for i in range(100):
    population2 = select_next_population(population_df(population2), 
                                         inverse=True)
    population2 = cross_population(population2, 
                                  probability=0.8, 
                                  allow_range=range(15, 46))
    population2 = mutate_population(population2, 
                                   probability=0.2, 
                                   allow_range=range(15, 46))

df2 = population_df(population2)
print(df2)
print(df2[df2['sel_probability'] == df2['sel_probability'].min()].head(1))

   chromosome  phenotype  fun_value  sel_probability
0      001111         15     -179.4         0.090909
1      001111         15     -179.4         0.090909
2      001111         15     -179.4         0.090909
3      001111         15     -179.4         0.090909
4      001111         15     -179.4         0.090909
5      001111         15     -179.4         0.090909
6      001111         15     -179.4         0.090909
7      001111         15     -179.4         0.090909
8      001111         15     -179.4         0.090909
9      001111         15     -179.4         0.090909
10     001111         15     -179.4         0.090909
  chromosome  phenotype  fun_value  sel_probability
0     001111         15     -179.4         0.090909
